In [8]:
import qiskit
from qiskit.test.ibmq_mock import mock_get_backend
from qiskit_aer import Aer
from qiskit_ibm_provider import IBMProvider

from mitiq import zne
from mitiq.interface.mitiq_qiskit.qiskit_utils import initialized_depolarizing_noise
from qiskit.visualization import plot_histogram

In [2]:
USE_REAL_HARDWARE = False

In [3]:
qreg, creg = qiskit.QuantumRegister(1), qiskit.ClassicalRegister(1)
circuit = qiskit.QuantumCircuit(qreg, creg)
for _ in range(10):
    circuit.x(qreg)
circuit.measure(qreg, creg)
print(circuit)

      ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌─┐
  q0: ┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤M├
      └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└╥┘
c0: 1/═══════════════════════════════════════════════════╩═
                                                         0 


In [4]:
if IBMProvider.saved_accounts() and USE_REAL_HARDWARE:
    provider = IBMProvider()
    backend = provider.get_backend("ibmq_qasm_simulator")  # Set quantum computer here!
else:
    # Default to a simulator.
    backend = Aer.get_backend("qasm_simulator"),


def ibmq_executor(circuit: qiskit.QuantumCircuit, shots: int = 8192) -> float:
    """Returns the expectation value to be mitigated.

    Args:
        circuit: Circuit to run.
        shots: Number of times to execute the circuit to compute the expectation value.
    """
    if USE_REAL_HARDWARE:
        # Run the circuit on hardware
        job = qiskit.execute(
            experiments=circuit,
            backend=backend,
            optimization_level=0,  # Important to preserve folded gates.
            shots=shots
        )
    else:
        # Simulate the circuit with noise
        noise_model = initialized_depolarizing_noise(noise_level=0.02)
        job = qiskit.execute(
            experiments=circuit,
            backend=Aer.get_backend("qasm_simulator"),
            noise_model=noise_model,
            basis_gates=noise_model.basis_gates,
            optimization_level=0,  # Important to preserve folded gates.
            shots=shots,
        )

    # Convert from raw measurement counts to the expectation value
    counts = job.result().get_counts()
    if counts.get("0") is None:
        expectation_value = 0.
    else:
        expectation_value = counts.get("0") / shots
    return expectation_value

In [10]:
unmitigated = ibmq_executor(circuit)
mitigated = zne.execute_with_zne(circuit, ibmq_executor)
print(f"Unmitigated result {unmitigated:.3f}")
print(f"Mitigated result {mitigated:.3f}")

Unmitigated result 0.908
Mitigated result 0.969


In [11]:
linear_factory = zne.inference.LinearFactory(scale_factors=[1.0, 1.5, 2.0, 2.5, 3.0])
mitigated = zne.execute_with_zne(circuit, ibmq_executor, factory=linear_factory)
print(f"Mitigated result {mitigated:.3f}")

Mitigated result 0.978


In [12]:
mitigated = zne.execute_with_zne(circuit, ibmq_executor, scale_noise=zne.scaling.fold_global)
print(f"Mitigated result {mitigated:.3f}")

Mitigated result 1.000


In [13]:
scale_factors = [1., 1.5, 2., 2.5, 3.]
folded_circuits = [
        zne.scaling.fold_gates_at_random(circuit, scale)
        for scale in scale_factors
]

# Check that the circuit depth is (approximately) scaled as expected
for j, c in enumerate(folded_circuits):
    print(f"Number of gates of folded circuit {j} scaled by: {len(c) / len(circuit):.3f}")

Number of gates of folded circuit 0 scaled by: 1.000
Number of gates of folded circuit 1 scaled by: 1.364
Number of gates of folded circuit 2 scaled by: 1.909
Number of gates of folded circuit 3 scaled by: 2.273
Number of gates of folded circuit 4 scaled by: 2.818


In [14]:
shots = 8192

if USE_REAL_HARDWARE:
    # Run the circuit on hardware
    job = qiskit.execute(
        experiments=folded_circuits,
        backend=backend,
        optimization_level=0,  # Important to preserve folded gates.
        shots=shots
    )
else:
    # Simulate the circuit with noise
    noise_model = initialized_depolarizing_noise(noise_level=0.05)
    job = qiskit.execute(
        experiments=folded_circuits,
        backend=Aer.get_backend("qasm_simulator"),
        noise_model=noise_model,
        basis_gates=noise_model.basis_gates,
        optimization_level=0,  # Important to preserve folded gates.
        shots=shots,
    )

In [15]:
all_counts = [job.result().get_counts(i) for i in range(len(folded_circuits))]
expectation_values = [counts.get("0") / shots for counts in all_counts]
print(f"Expectation values:\n{expectation_values}")

Expectation values:
[0.793701171875, 0.74658203125, 0.6827392578125, 0.640380859375, 0.6026611328125]


In [19]:
print("Unmitigated expectation value:", round(expectation_values[0], 3))

Unmitigated expectation value: 0.794


In [20]:
zero_noise_value = zne.ExpFactory.extrapolate(scale_factors, expectation_values, asymptote=0.5)
print(f"Extrapolated zero-noise value:", round(zero_noise_value, 3))

Extrapolated zero-noise value: 1.012
